In [1]:
import xgboost as xgb
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
%matplotlib inline 
import scipy.stats as ss 
import matplotlib.style as ms
from time import time
#ms.use('fivethirtyeight')

In [ ]:
order_path = r'../monthly_bird/'
out_path = r'../models/'
os.chdir(order_path)
for i in os.listdir():
    filetype = os.path.splitext(i)[1]
    filename = os.path.splitext(i)[0]
    
    if filetype == '.csv':
        print(filename)
        data = pd.read_csv(i)
        data = data.drop(columns=['date','lon','lat','individual','source file','month'])#
        y = data.iloc[:,0]
        X = data.iloc[:,1:]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 100)
        dtrain = xgb.DMatrix(data=X_train,label=y_train)
        dtest = xgb.DMatrix(data=X_test,label=y_test)
        watchlist = [(dtrain, 'train'),(dtest, 'eval')]
        base_y = y_train.mean()
        
        params = {'eta':0.01, 
                  'gamma':0.4,
                  'max_depth':8,
                  'min_child_weight':4,
                  #'silent':2, 
                  'verbosity':3,
                  'objective':'binary:logistic',
                  'subsample':0.8,
                  'eval_metric':'auc',
                  'reg_lambda':0.1,
                  'colsample_bytree':0.8
          }

        num_round = 1000
        xgb_model = xgb.train(params,                   # set tuning parameters from above                   
                              dtrain,                   # training data
                              num_round,                # maximum of 1000 iterations (trees)
                              evals=watchlist,          # use watchlist for early stopping 
                              early_stopping_rounds=50, # stop after 50 iterations (trees) without increase in AUC
                              verbose_eval=True)        # display iteration progress
        xgb_model.save_model(out_path+filename+'.json')
        #break